# Milestone 2 - Data gathering and preprocessing

## 1. Narrowing down the research question

### Research Questions

In this research project we will try to analyse a corpus of popular songs to try to identify  chords differences between verses and choruses. We will try to  answer the following research questions :
<ol>
<li>Does the chord distribution of the choruses differ from the one in the verses?</li>
<li>Is there a different chord sequence distribution in the choruses compared to the verses?</li>
<li>How are these distributions evolving over time? </li>
</ol>

In the first question we want to know if a difference exists in terms of chords statistic. Are we more likely to find a specific chord in the choruses? Are less different chords used in the chorus compared to the other parts of a song? <br> 
For the second one we want to have a more melodic insight. Can we find specific patterns? Is the Markov model derived from the chorus different from its counterparts? <br>
In the third one the focus is on the time dimension. We want to know if the results of the two previous answers change over time. Were chorus closer to verses in 1968 than in 1985? How does each chord evolve in relation to the others? Do we find a sequence that appeared while other disappeared? 

These are all the underlying questions we want to answer under our main research questions.  

These questions relate to our original idea to fully characterize a chorus over the years, especially in comparison with verses. We now want to focus specifically on the chords to differentiate these different parts, while keeping the temporal dimension as a potential factor to observe changes.

### Dataset presentation

To give us the means to answer our questions, we have selected a dataset containing approximately 900 Pop-Rock songs in the top Billboard charts from the 60s to the 90s. They are simple text files with the following informations : 
* release date
* song title 
* artist name
* labels for the different parts of the song (such as chorus or verse)
* timestamp of each musical phrase beginning
* chords

### Procedure

We will use the chorus/verse annotations to classify the chords in each group. This will allow us to divide the chords in two groups and compute statistics and distributions for each of them. As discussed in the research questions, we will start with a basic characterization, simply comparing which chords appear in which section. We will then move deeper and compare the distributions of chords as well as the Markov models. These will be computed at least with bigrams, perhaps with higher-n n-grams depending on the number of chords in each section (as it makes little sense to use n-grams with n close to the number of chords in a given section).

The metadata, especially the release date, will be used to study the evolution over time of the previously discussed statistics. Depending on the distribution of songs over the years, time analysis will be discussed either over years or over decades.

### Possible outcomes and confidence measures

The different outcomes we can reasonably expect are: 
<ul>
    <li> <strong>Null results:</strong> There are no significant differences between choruses and verses and no evolution over time. This could be explained by a bias in the corpus toward a specific Pop-Rock genre using the same chords all the time or maybe there is indeed no difference in the chords used in a chorus and the ones used in verses, which would constitute an answer for our questions. </li>
<li> <strong>Narrow chorus chord distribution:</strong> Since the chorus has to be immediately recognized as one, maybe the composers make more extensive use of a sub group of the chords to ensure it. The same reasoning could be applied to the chord sequences: perhaps some specific ones will be more dominant in the chorus.</li>
<li> <strong>Temporal evolution:</strong> It will be interesting to see if the differences between verse and chorus change over the years. This evolution, if present, could be linear or oscillating. A linear narrowing would imply that choruses are becoming more and more similar or verse more and more diverse. An oscillating pattern would be interesting as musical phenomenon could appear and disappear.</li>
</ul>

Statistical tests will be used throughtout our analysis to check if our findings are statistically relevant. Error bars will also be included in all our graphics to avoid wrong conclusions. Of course the relative small size of our corpus will influence our results but only further analysis can reveal if significant results can still be found.

## 2. Gathering the data
The dataset has been created by [1] and corresponds to a random sample of 890 Billboard chart slots presented at ISMIR 2011 and MIREX 2012. Due to the nature of the sampling algorithm, there are some duplicates and this results in only 740 distinct songs. According to the authors, training algorithms that assume independent, identically distributed data should retain the duplicates.<br> This dataset is publicly available at https://ddmal.music.mcgill.ca/research/The_McGill_Billboard_Project_(Chord_Analysis_Dataset)/ and can be downloaded in various formats. Different features are given by the authors. In this project we will use metadata and chords annotations. 
The first dataset used is the index to the dataset (csv format), containing the following fields:
<ul>
<li><b>id</b>, the index for the sample entry;</li>
<li><b>chart_date</b>, the date of the chart for the entry;</li>
<li><b>target_rank</b>, the desired rank on that chart;</li>
<li><b>actual_rank</b>, the rank of the song actually annotated, which may be up to 2 ranks higher or lower than the target rank [1, 2];</li>
<li><b>title</b>, the title of the song annotated;</li>
<li><b>artist</b>, the name of the artist performing the song annotated;</li>
<li><b>peak_rank</b>, the highest rank the song annotated ever achieved on the Billboard Hot 100; and</li>
<li><b>weeks_on_chart</b>, the number of weeks the song annotated spent on the Billboard Hot 100 chart in total.</li>
</ul>

The main dataset comprehends chords, structure, instrumentation, and timing, given in a txt format. The annotation for each song begins with a header containing the title of the song, the name of the artist, the metre and the tonic pitch class of the opening key. In the main body, each line consists of a single phrase and begins with its timestamp, followed by the chords. This requires us to design a specific parser, as will be discussed in the next section.

[1]: John Ashley Burgoyne, Jonathan Wild, and Ichiro Fujinaga, ‘An Expert Ground Truth Set for Audio Chord Recognition and Music Analysis’, in Proceedings of the 12th International Society for Music Information Retrieval Conference, ed. Anssi Klapuri and Colby Leider (Miami, FL, 2011), pp. 633–38

## 3. Data format

The goal of this question is to load the data and have a look at it. A specific parser is designed to do this automatically, in order to extract and store in a Pandas dataframe all the relevant informations and musical features.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

### Metadata

In [ ]:
metadata_df = pd.read_csv("data/billboard-2.0-index.csv")

In [ ]:
print('There are %d entries in the index table.' %len(metadata_df))

In [ ]:
print('There are %d entries with a given title.' %metadata_df.title.isna().value_counts()[0])

In [ ]:
print('There are %d entries with a given artist.' %metadata_df.artist.isna().value_counts()[0])

In [ ]:
print('There are %d entries with a given chart date.' %metadata_df.chart_date.isna().value_counts()[0])

In [ ]:
months = {'01':'January', 
          '02':'February',
          '03':'March',
          '04':'April',
          '05':'May',
          '06':'June',
          '07':'July',
          '08':'August',
          '09':'September',
          '10':'October',
          '11':'November',
          '12':'December'}
def format_date(date):
    year = date[:4]
    month = date[5:7]
    day = date[-2:]
    if day == '01':
        suffix = 'st'
    elif day == '02':
        suffix = 'nd'
    elif day == '03':
        suffix = 'rd'
    else:
        suffix = 'th'
        
    if day[0] == '0':
        day = day[1]
    
    date_string = months[month] + ' ' + day + suffix + ', ' + year
    return(date_string)

#Test
format_date('1958-08-04')

In [ ]:
print('The songs range from %s to %s.' %(format_date(metadata_df.chart_date.min()), format_date(metadata_df.chart_date.max())))

Let's analyse the song distribution over the year.

In [ ]:
metadata_df['year'] = metadata_df['chart_date'].map(lambda date: int(date[:4]))
n_bins = metadata_df.year.max() - metadata_df.year.min() + 1
metadata_df.year.hist(bins = n_bins)
plt.title('Song distribution over the years', fontsize = 20)
plt.xlabel('Year', fontsize = 18)
plt.ylabel('Number of songs', fontsize = 18)
plt.show()

In [ ]:
min_number_songs = metadata_df.groupby(['year']).count().id.min()
min_number_songs_year = metadata_df.groupby(['year']).count().id.idxmin()
max_number_songs = metadata_df.groupby(['year']).count().id.max()
max_number_songs_year = metadata_df.groupby(['year']).count().id.idxmax()
mean_number_songs = metadata_df.groupby(['year']).count().id.mean()

print('Year with the minimum number of songs: %d (with %d songs)' %(min_number_songs_year, min_number_songs))
print('Year with the maximum number of songs: %d (with %d songs)' %(max_number_songs_year, max_number_songs))
print('The average number of songs per year is %0.2f.' %mean_number_songs)

Let's now have a look at the artists.

In [ ]:
number_artist = len(metadata_df.artist.unique()) - 1 #The -1 is to take into account the missing values in the dataframe
print('There are %d different artists in the dataframe.' %number_artist)

In [ ]:
min_number_songs_per_artist = metadata_df.groupby(['artist']).count().id.min()
artist_min_songs = metadata_df.groupby(['artist']).count().id.idxmin()
max_number_songs_per_artist = metadata_df.groupby(['artist']).count().id.max()
artist_max_songs = metadata_df.groupby(['artist']).count().id.idxmax()
#Since the distribution is highly skewed, we use the median instead of the mean here
median_songs_per_artist = metadata_df.groupby('artist').count().id.median() 
artist_with_5songs = metadata_df.groupby('artist').count().id.map(lambda x: x >= 5).value_counts()[1]

print('Artist with the lowest number of songs: %s (with %d songs)' %(artist_min_songs, min_number_songs_per_artist))
print('Artist with the highest number of songs: %s (with %d songs)' %(artist_max_songs, max_number_songs_per_artist))
print('The median number of songs per artist is %0.2f.' %median_songs_per_artist)
print('Only %d artists have at least 5 songs in the corpus.' %artist_with_5songs)

As we can see with the distribution of the number of songs per artist, there are clearly some artist (such as Elvis Presley) with more than 10 songs in the corpus but the majority of them only has one or two songs. This results in a highly skewed distribution.

In [ ]:
metadata_df.groupby('artist').count().id.hist(bins = max_number_songs_per_artist - min_number_songs_per_artist + 1)
plt.title('Distribution of the number of songs per artist', fontsize = 20)
plt.ylabel('Number of artists', fontsize = 18)
plt.xlabel('Number of songs', fontsize = 18)
plt.show()

In [ ]:
#Note that the names of the artist are not easily readable (due to the high amount of different artists in the corpus).
#This graph is only meant to show the diversity in the number of songs per artist, not precise information for each artist
plt.figure(figsize=(18,10))
metadata_df.groupby('artist').count().id.plot(kind = 'bar')
plt.title('Number of songs for each artist', fontsize = 20)
plt.xlabel('Artist', fontsize = 18)
plt.ylabel('Number of songs', fontsize = 18)
plt.xticks(fontsize = 6)
plt.show()

### Parser 

In [ ]:
SONG_ID, LINE_NUMBER, MEASURE_NUMBER, CHORD_NUMBER, SEQUENCE_NUMBER, \
CHORD, INSTRUMENT, TYPE, TIME, STRUCTURE, DURATION, REPETITION, ELID = \
"song_id","line_id", "measure_id", "chord_id", "sequence_id",\
"chord", "instrument", "section_type", "time", "section_structure", "duration", "repetition", "elided"

#This is dependant of "metre" in the txt files.
METRE = "metre"

#Create a new dictionary from two other
def immutable_merge(dic1, dic2):
    result = dic1.copy()
    result.update(dic2)
    return result

#Create a row of the futur df as a dictionary
def create_row(persistent_attributes, line_attributes,
               measure_number = None, chord_number = None, chord = None, duration = None):
    result = immutable_merge(persistent_attributes, line_attributes)
    
    if not (measure_number is None and measure_number is None and chord_number is None and duration is None):
        result[MEASURE_NUMBER] = measure_number
        result[CHORD_NUMBER] = chord_number
        result[CHORD] = chord
        result[DURATION] = duration
    
    return result

#Generate the attributes of a given line and update the sequence counter
def process_line_metadata(header, line_counter, old_line_attributes, sequence_counter, suffix = ""):
    
    result = {}
    
    #Suffix (main instrument, elid, repetition)
    old_instrument = str(old_line_attributes.get(INSTRUMENT))
    
    for suffix in suffix.split(", "):
        
        suffix = suffix.strip()
        
        #Repetition
        if re.match("^x\d+$",suffix):
            result[REPETITION] = int(suffix[1])
        
        #Elid
        elif suffix == "->":
            result[ELID] = True

        #Instrument
        else:
            ##New instrument
            if len(suffix) > 0 and suffix != "\n":
                result[INSTRUMENT] = suffix.strip("\n").strip(",").strip()

            ##Main instrument continued (experimental)
            elif not old_instrument.endswith(")") and old_instrument.lower() not in ["nan","none"] \
            and len(old_instrument)>0:
                result[INSTRUMENT] = old_instrument.strip("(")

        
    #Line number
    result[LINE_NUMBER] = line_counter

    
    #Header    
    header_items = header.split()
        
    result[TIME] = header_items[0]
    
    #Case where a section is continued
    if len(header_items) == 1:
        result[TYPE] = old_line_attributes.get(TYPE)
        result[STRUCTURE] = old_line_attributes.get(STRUCTURE)
        result[SEQUENCE_NUMBER] = old_line_attributes.get(SEQUENCE_NUMBER)
    
    #Case where a section has no structure (silence, end, fadeout)
    elif len(header_items) == 2:
        
        #Z is a structure, not a type.
        if header_items[1].strip().strip(",") == "Z":
            result[STRUCTURE] = header_items[1].strip().strip(",")
        else:
            result[TYPE] = header_items[1].strip().strip(",")
            
        result[SEQUENCE_NUMBER] = sequence_counter
        sequence_counter += 1
    
    #Case where a section begins.
    elif len(header_items) == 3:
        result[STRUCTURE] = header_items[1].strip().strip(",")
        result[TYPE] = header_items[2].strip().strip(",")
        result[SEQUENCE_NUMBER] = sequence_counter
        sequence_counter += 1
    
    return sequence_counter, result

In [ ]:
def parse_song_to_dict(song_id, path):
    
    rows = []
    persistent_attributes = {}
    
    persistent_attributes[SONG_ID] = song_id
    
    with open(path,"r") as file:
        line = file.readline()
        
        line_counter = 0
        measure_counter = 0
        chord_counter = 0
        sequence_counter = 0
        line_attributes = {}
        old_chord = None
 

        while line:
        
            if line != "\n":

                #Attribute lines
                if line.startswith("#"):
                    attribute, value = line.strip("#").split(":",1)
                    persistent_attributes[attribute.strip(" ")] = value.strip(" ").strip("\n")

                else:
                    line_items = line.split("|")

                    #Special lines
                    if len(line_items) <= 1:
                        sequence_counter, line_attributes = \
                        process_line_metadata(line, line_counter, line_attributes, sequence_counter)
                        row = create_row(persistent_attributes, line_attributes)
                        rows.append(row)

                    #Standard lines    
                    else:                    
                        header = line_items[0]
                        suffix = line_items[-1]
                        measures = line_items[1:-1]

                        sequence_counter, line_attributes = \
                        process_line_metadata(header, line_counter, line_attributes, sequence_counter, suffix)  

                        for measure in measures:
                            
                            chords = measure.split()
                            
                            #Special metric (experimental)
                            old_metre = persistent_attributes.get(METRE)
                            if re.match("^\(\d/\d\)$", chords[0]):
                                persistent_attributes[METRE] = str(chords[0][1]) + "/" + str(chords[0][3])
                                chords = chords[1:]
                            
                            if len(chords) == 1:
                                duration = "measure"
                            else:
                                duration = "beat"
                            
                            for chord in chords:
                                
                                if chord == ".":
                                    chord = old_chord
                                
                                row = create_row(persistent_attributes, line_attributes,
                                                 measure_counter, chord_counter, chord, duration)
                                rows.append(row)
                                old_chord = chord
                                chord_counter += 1

                            measure_counter += 1
                            persistent_attributes[METRE] = old_metre
            
            #Finally
            line_counter += 1
            line = file.readline()
    
    
    return rows

In [ ]:
test = pd.DataFrame(parse_song_to_dict(0,"data/McGill-Billboard/0004/salami_chords.txt"))

In [ ]:
def create_whole_collection_df():
    
    path = "data/McGill-Billboard/"
    file_name = "/salami_chords.txt"
    UPPER_BOUND = 1300
    
    whole_collection = []
    
    i = 0
    while i <= UPPER_BOUND:
        full_path = path + "0"*(4-len(str(i)))+ str(i) + file_name
        
        if os.path.exists(full_path):
            whole_collection += parse_song_to_dict(i, full_path)
        
        i += 1
        
    whole_collection_df = pd.DataFrame(whole_collection)
    
    return whole_collection_df.astype({SEQUENCE_NUMBER: 'Int64', MEASURE_NUMBER: 'Int64', CHORD_NUMBER: 'Int64', \
                                      REPETITION: 'Int64'})

In [ ]:
collection_df = create_whole_collection_df()

In [ ]:
collection_df.sample(10)

In [ ]:
#BUGS: Point = répétition du même accord? A élucider et modifier. DONE
# Il y a des mesures à un seul accord, d'autres à un accord par temps. A prendre en compte ! DONE
#Introduire un sequence_id qui identifie un chorus, un verse, etc... DONE
#Signalétique indiquant une répétition (x) ou un elid (->) à prendre en compte (colonne instrument) DONE
#Dans certains morceaux, il y a des mesures au metre différent du metre principal, indiqué entre (). DONE
#La lettre Z est parfois classée comme "type", parfois comme "structure" DONE

#&pause est de taille arbitraire, pas forcément une mesure

Let's now analyse some basic statistics on our corpus.

In [ ]:
#Basic statistics: Ten most frequent chords in the corpus
collection_df.groupby(CHORD)[CHORD].count().sort_values(ascending = False).head(10).plot.bar()
plt.title('Ten most frequent chords in the corpus', fontsize = 20)
plt.xlabel('Chord', fontsize = 18)
plt.xticks(rotation = 'horizontal', fontsize = 9)
plt.ylabel('Counts')
plt.show()

In [ ]:
#Basic statistics: Number of unique chords per songs
unique_chord_songs = collection_df[[SONG_ID,CHORD]].drop_duplicates().groupby(SONG_ID).count()
n_bins = int(unique_chord_songs.max())+1
unique_chord_songs.plot.hist(bins = n_bins, legend = False)
plt.title('Distribution of the number of unique chords per song', fontsize = 20)
plt.xlabel('Number of unique chord', fontsize = 18)
plt.ylabel('Frequency', fontsize = 18)
plt.show()

In [ ]:
print("Number of songs that contain a chorus : %s" %str(np.sum(collection_df[[SONG_ID,TYPE]].drop_duplicates()[TYPE] == "chorus")))

print("Number of songs that contain a verse : %s" %str(np.sum(collection_df[[SONG_ID,TYPE]].drop_duplicates()[TYPE] == "verse")))

unique_section_songs = collection_df[[SONG_ID,TYPE]].drop_duplicates().groupby(SONG_ID)[TYPE].apply(list)
print("Number of songs that contain both : %s" %str(np.sum(unique_section_songs.apply(lambda l : "chorus" in l and "verse" in l))))